## **Segmenting and Clustering Neighborhoods in Toronto**
Peer Review Assignment

In [1]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [2]:
#extract data from Wikipedia
import requests
wikipedia_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
raw_wikipedia_page = requests.get(wikipedia_link).text

In [3]:
# import the beautifulsoup library
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_wikipedia_page,'lxml')
toronto_data = BeautifulSoup(raw_wikipedia_page, 'lxml')

In [4]:
# create a new DataFrame
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [5]:
# clean the dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df_toronto = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_toronto.head()

,index,Neighborhood


In [6]:
import csv
import json
import xml

In [7]:
# Get the latitude and longitude coordinates
filename = "http://cocl.us/Geospatial_data"
headers = ["PostalCode","Latitude","Longitude"]

df_ll = pd.read_csv(filename, names = headers)
df_ll.head()

,PostalCode,Latitude,Longitude
0,Postal Code,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174


In [8]:
# Concat the two dataframes

In [9]:
df_final =pd.concat([df_toronto, df_ll], axis=1)
df_final.head()

,index,Neighborhood,PostalCode,Latitude,Longitude
0,NaN,NaN,Postal Code,Latitude,Longitude
1,NaN,NaN,M1B,43.8066863,-79.1943534
2,NaN,NaN,M1C,43.7845351,-79.1604971
3,NaN,NaN,M1E,43.7635726,-79.1887115
4,NaN,NaN,M1G,43.7709921,-79.2169174


In [10]:
del df_final['PostalCode']
df_final.head()

,index,Neighborhood,Latitude,Longitude
0,NaN,NaN,Latitude,Longitude
1,NaN,NaN,43.8066863,-79.1943534
2,NaN,NaN,43.7845351,-79.1604971
3,NaN,NaN,43.7635726,-79.1887115
4,NaN,NaN,43.7709921,-79.2169174


In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  25.21 MB/s
geopy-1.19.0-p 100% |################################| Time: 0:00:00  36.64 MB/s
Fetching package metadata .............
Solving package specifications: 

In [ ]:
# find the geographical coordinates of Toronto

address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# Assign column names to map_TO.
map_TO = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(map_TO['Latitude'], map_TO['Longitude'], map_TO['Borough'], map_TO['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

In [ ]:
CLIENT_ID = 'W2QHU2K2ZV0TYRS1LZN1XRNPZROAAP3VILHKFOIDS5EYD03Q' # your Foursquare ID
CLIENT_SECRET = 'LIKTZQJNCJKPPSPSGTU4HRZ3G151UI2ENHYHZMYJQBCVAYIW' # your Foursquare Secret
VERSION = '20190330' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:

toronto = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )